In [9]:
import os
import glob

import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [10]:
train=pd.read_json('res/train.json',typ='frame')

In [11]:
train

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000


In [17]:
data='like_cnt > = 3'
song_id_filtered_df = train.query(data)
song_id_filtered_df=song_id_filtered_df[['id','tags','songs']]
song_id_filtered_df

,id,tags,songs
0,61281,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
2,76951,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,147456,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,27616,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."
5,69252,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...","[418694, 222305, 96545, 135950, 304687, 457451..."
...,...,...,...
115066,120325,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]","[429629, 441511, 612106, 516359, 691768, 38714..."
115067,106976,[일렉],"[321330, 216057, 534472, 240306, 331098, 23288..."
115068,11343,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]","[50512, 249024, 250608, 371171, 229942, 694943..."
115069,131982,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]","[533534, 608114, 343608, 417140, 609009, 30217..."


In [18]:
# 플레이리스트 아이디(id)와 수록곡(songs) 추출
plylst_song_map = song_id_filtered_df[['id', 'songs']]
# unnest songs
plylst_song_map_unnest = np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))), 
        np.concatenate(plylst_song_map.songs.values)
    )
)

# unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['id'] = plylst_song_map['id'].astype(str)
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)

# unnest 객체 제거
del plylst_song_map_unnest

In [19]:
plylst_song_map

,id,songs
0,61281,525514
1,61281,129701
2,61281,383374
3,61281,562083
4,61281,297861
...,...,...
4276188,100389,111365
4276189,100389,51373
4276190,100389,640239
4276191,100389,13759


In [20]:
unique_songs=plylst_song_map['songs'].drop_duplicates()
unique=unique_songs.to_frame()
type(unique)
#list_unique=list(unique.value)
list_unique=np.array(unique['songs'].tolist())
#for i in range(len(unique)):
#    print(unique[i])
len(list_unique)

560735

In [ ]:
song_tags=[]
for i in range(len(unique)):
    song=plylst_song_tag_map[plylst_song_tag_map['songs']==list_unique[i]]
    x=mapping(song)
    song_tags.append(x)
    print(i)
    #duplicates = dframe[dframe.duplicated()].drop_duplicates('tags')
song_tags_df= pd.DataFrame(song_tags)
song_tags_matrix=pd.merge(unique,song_tags_df)
print(song_tags_matrix)

In [14]:
m=pd.concat([val_id_songs, songs], axis=1)
m

,id,songs,len_song
0,61281,"[525514, 129701, 383374, 562083, 297861, 13954...",19.0
1,10532,"[432406, 675945, 497066, 120377, 389529, 24427...",42.0
2,76951,"[83116, 276692, 166267, 186301, 354465, 256598...",28.0
3,147456,"[394031, 195524, 540149, 287984, 440773, 10033...",38.0
4,27616,"[159327, 553610, 5130, 645103, 294435, 100657,...",53.0
...,...,...,...
115066,120325,"[429629, 441511, 612106, 516359, 691768, 38714...",NaN
115067,106976,"[321330, 216057, 534472, 240306, 331098, 23288...",NaN
115068,11343,"[50512, 249024, 250608, 371171, 229942, 694943...",NaN
115069,131982,"[533534, 608114, 343608, 417140, 609009, 30217...",NaN


In [7]:
data='len_song > =1'
song_id_filtered_df = m.query(data)
song_id_filtered_df=song_id_filtered_df[['id','songs']]